In [2]:
import os
import numpy as np
import pandas as pd
from scipy.io.wavfile import write

# Ensure directories exist
os.makedirs('outputs/dasham_ansh', exist_ok=True)
os.makedirs('outputs/shata_ashta', exist_ok=True)

# Read EEG data
data = pd.read_csv('data/eeg_data.csv', low_memory=False)
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
data.set_index('TimeStamp', inplace=True)

# Combine EEG bands
data['Delta'] = data[['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10']].sum(axis=1)
data['Theta'] = data[['Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10']].sum(axis=1)
data['Alpha'] = data[['Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10']].sum(axis=1)
data['Beta'] = data[['Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10']].sum(axis=1)
data['Gamma'] = data[['Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10']].sum(axis=1)

# Constants
dasham_duration = 27  # Dasham Ansh duration (seconds)
shata_ashta_duration = 108  # Shata-Ashta Patha duration (seconds)
sample_rate = 44100  # Sample rate for audio

# Shruti frequencies
shruti_frequencies = [
    256, 266, 276, 288, 300, 312, 324, 336, 348, 362, 
    376, 390, 404, 420, 436, 452, 470, 488, 508, 528, 
    548, 568
]

# Helper function: Generate audio
def generate_audio(data, output_folder, audio_type, channel_name, duration):
    print(f"Generating {audio_type} for {channel_name} in {output_folder}...")
    os.makedirs(output_folder, exist_ok=True)
    data_min, data_max = data.min(), data.max()

    # Map data to Shrutis
    xp = np.linspace(data_min, data_max, len(shruti_frequencies))
    mapped_shrutis = np.interp(data, xp, shruti_frequencies)

    audio_data = np.zeros(int(sample_rate * duration))
    time_values = np.linspace(0, duration, int(sample_rate * duration))

    if audio_type == "Nada":  # Single-tone flow
        for i, freq in enumerate(mapped_shrutis):
            audio_data += 0.3 * np.sin(2 * np.pi * freq * time_values)  # Basic tone
            audio_data += 0.1 * np.sin(2 * np.pi * (freq + 5) * time_values)  # Subtle harmonic
            if i % 100 == 0:  # Smooth transitions every 100 steps
                audio_data += 0.05 * np.sin(2 * np.pi * (freq - 3) * time_values)

    elif audio_type == "Anahata":  # Resonant chords
        for i, freq in enumerate(mapped_shrutis):
            if i + 3 < len(mapped_shrutis):
                chord = mapped_shrutis[i:i+3]
                for c in chord:
                    audio_data += 0.2 * np.sin(2 * np.pi * c * time_values)
                audio_data += 0.05 * np.sin(2 * np.pi * (freq + 50) * time_values)  # High harmonic

    elif audio_type == "Brahmanda":  # Pulsing rhythm
        for i, freq in enumerate(mapped_shrutis):
            pulse = 0.5 * (1 + np.sin(2 * np.pi * 0.2 * time_values))  # Pulsing amplitude
            audio_data += pulse * np.sin(2 * np.pi * freq * time_values)
            audio_data += 0.1 * np.sin(2 * np.pi * (freq * 1.5) * time_values)  # High-frequency hum

    # Normalize audio
    audio_data /= np.max(np.abs(audio_data)) if np.max(np.abs(audio_data)) != 0 else 1
    output_path = f"{output_folder}/{channel_name}_{audio_type}.wav"
    write(output_path, sample_rate, audio_data.astype(np.float32))
    print(f"Saved {audio_type} audio for {channel_name} to {output_path}")

# Generate Dasham Ansh
for ansh_idx in range(3):  # Limited for testing
    ansh_folder = f"outputs/dasham_ansh/ansh_{ansh_idx + 1}"
    sampled_data = data.sample(frac=0.108, random_state=ansh_idx)
    for band in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']:
        for audio_type in ["Nada", "Anahata", "Brahmanda"]:
            generate_audio(sampled_data[band], ansh_folder, audio_type, band, dasham_duration)

# Generate Shata-Ashta Patha
for band in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']:
    for audio_type in ["Nada", "Anahata", "Brahmanda"]:
        generate_audio(
            data[band].sample(frac=(shata_ashta_duration / total_duration), random_state=42),
            "outputs/shata_ashta_patha",
            audio_type,
            band,
            shata_ashta_duration,
        )

print("\nAudio files generated successfully!")


Generating Nada for Delta in outputs/dasham_ansh/ansh_1...
Saved Nada audio for Delta to outputs/dasham_ansh/ansh_1/Delta_Nada.wav
Generating Anahata for Delta in outputs/dasham_ansh/ansh_1...
Saved Anahata audio for Delta to outputs/dasham_ansh/ansh_1/Delta_Anahata.wav
Generating Brahmanda for Delta in outputs/dasham_ansh/ansh_1...
Saved Brahmanda audio for Delta to outputs/dasham_ansh/ansh_1/Delta_Brahmanda.wav
Generating Nada for Theta in outputs/dasham_ansh/ansh_1...
Saved Nada audio for Theta to outputs/dasham_ansh/ansh_1/Theta_Nada.wav
Generating Anahata for Theta in outputs/dasham_ansh/ansh_1...
Saved Anahata audio for Theta to outputs/dasham_ansh/ansh_1/Theta_Anahata.wav
Generating Brahmanda for Theta in outputs/dasham_ansh/ansh_1...
Saved Brahmanda audio for Theta to outputs/dasham_ansh/ansh_1/Theta_Brahmanda.wav
Generating Nada for Alpha in outputs/dasham_ansh/ansh_1...
Saved Nada audio for Alpha to outputs/dasham_ansh/ansh_1/Alpha_Nada.wav
Generating Anahata for Alpha in out

In [ ]:
# Calculate the total recording duration from timestamps
timestamps = pd.to_datetime(data.index)
total_duration = (timestamps.max() - timestamps.min()).total_seconds()
print(f"Total Recording Duration: {total_duration} seconds")


In [5]:
import os
import numpy as np
from scipy.io.wavfile import write

# Constants
sample_rate = 44100  # Sample rate for audio
shata_ashta_duration = 108  # Duration for Shata Ashta Patha files (in seconds)
shruti_frequencies = [
    256, 266, 276, 288, 300, 312, 324, 336, 348, 362,
    376, 390, 404, 420, 436, 452, 470, 488, 508, 528,
    548, 568
]

# Function to generate combined Chalan Swara audio
def generate_combined_chalan_swara(data_dict, output_folder, duration):
    print(f"Generating Combined Chalan Swara in {output_folder}...")
    os.makedirs(output_folder, exist_ok=True)

    # Initialize audio data
    combined_audio = np.zeros(int(sample_rate * duration))
    time_values = np.linspace(0, duration, int(sample_rate * duration))

    # Process each channel
    for channel_name, data in data_dict.items():
        print(f"Processing channel: {channel_name}")
        data_min, data_max = data.min(), data.max()

        # Map data to Shruti frequencies
        xp = np.linspace(data_min, data_max, len(shruti_frequencies))
        mapped_shrutis = np.interp(data, xp, shruti_frequencies)

        glide_factor = np.linspace(0, 1, len(mapped_shrutis))  # Smooth glide effect
        channel_audio = np.zeros(int(sample_rate * duration))

        # Generate channel-specific audio
        for i, freq in enumerate(mapped_shrutis):
            dynamic_freq = freq + 10 * np.sin(2 * np.pi * glide_factor[i] * time_values)  # Micro-fluctuations
            channel_audio += 0.3 * np.sin(2 * np.pi * dynamic_freq * time_values)

        # Add to combined audio
        combined_audio += channel_audio

    # Normalize combined audio
    combined_audio /= np.max(np.abs(combined_audio)) if np.max(np.abs(combined_audio)) != 0 else 1
    output_path = os.path.join(output_folder, "Chalan_Swara.wav")
    write(output_path, sample_rate, combined_audio.astype(np.float32))
    print(f"Saved Combined Chalan Swara to {output_path}")

# Process Shata Ashta Patha folder
shata_ashta_folder = "outputs/shata_ashta"
new_chalan_swara_folder = "outputs/chalan_swara"

# Collect data for each channel
channel_data = {}
for audio_file in os.listdir(shata_ashta_folder):
    if audio_file.endswith(".wav") and "Chalan_Swara" not in audio_file:  # Skip existing Chalan Swara files
        # Extract channel name (e.g., "Alpha_Nada.wav" -> "Alpha")
        channel_name = audio_file.split("_")[0]
        # Load corresponding data (you can replace this with actual EEG data if available)
        channel_data[channel_name] = np.random.rand(1000)  # Replace with actual data if needed

# Generate the combined audio file
generate_combined_chalan_swara(channel_data, new_chalan_swara_folder, shata_ashta_duration)


Generating Combined Chalan Swara in outputs/chalan_swara...
Processing channel: Beta
Processing channel: Gamma
Processing channel: Theta
Processing channel: Delta
Processing channel: Alpha
Saved Combined Chalan Swara to outputs/chalan_swara/Chalan_Swara.wav


In [8]:
import os
import numpy as np
import pandas as pd
from scipy.io.wavfile import write

# Constants
sample_rate = 44100  # Sample rate for audio
chant_duration = 27  # Duration for each chant file (in seconds)
harmonic_multiplier = [1, 1.5, 2]  # Multipliers for harmonic layers
chant_pattern = [1, 1.25, 1.5, 1.25]  # Pattern for chanting oscillations

# Helper function to generate Vedic chanting audio
def generate_channel_chant(data, output_folder, channel_name, duration):
    print(f"Generating Vedic Chant for {channel_name} in {output_folder}...")
    os.makedirs(output_folder, exist_ok=True)

    # Map channel data to base frequency
    data_min, data_max = data.min(), data.max()
    base_frequency = np.interp(np.mean(data), (data_min, data_max), (136.1, 300))  # Dynamic base frequency (OM to high)

    # Time array for the duration
    time_values = np.linspace(0, duration, int(sample_rate * duration))
    
    # Chanting pattern tones
    chant_wave = np.zeros(len(time_values))
    syllable_duration = duration / len(chant_pattern)
    for i, multiplier in enumerate(chant_pattern):
        start = int(i * syllable_duration * sample_rate)
        end = int((i + 1) * syllable_duration * sample_rate)
        chant_wave[start:end] = np.sin(2 * np.pi * (base_frequency * multiplier) * time_values[start:end])
    
    # Add harmonic drone
    harmonic_wave = np.zeros(len(time_values))
    for multiplier in harmonic_multiplier:
        harmonic_wave += np.sin(2 * np.pi * (base_frequency * multiplier) * time_values)

    # Combine chant and harmonics
    combined_wave = chant_wave + 0.3 * harmonic_wave  # Chant louder than drone
    combined_wave /= np.max(np.abs(combined_wave)) if np.max(np.abs(combined_wave)) != 0 else 1

    # Save the chant as a WAV file
    file_name = f"{channel_name}_mantra.wav"
    output_path = os.path.join(output_folder, file_name)
    write(output_path, sample_rate, combined_wave.astype(np.float32))
    print(f"Saved Vedic Chant for {channel_name} to {output_path}")

# Read EEG data
data = pd.read_csv('data/eeg_data.csv', low_memory=False)
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
data.set_index('TimeStamp', inplace=True)

# Combine EEG bands
data['Delta'] = data[['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10']].sum(axis=1)
data['Theta'] = data[['Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10']].sum(axis=1)
data['Alpha'] = data[['Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10']].sum(axis=1)
data['Beta'] = data[['Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10']].sum(axis=1)
data['Gamma'] = data[['Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10']].sum(axis=1)

# Generate Vedic chants for each channel
output_folder = "outputs/shruti_mantra"
for channel in ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']:
    generate_channel_chant(data[channel], output_folder, channel, chant_duration)


Generating Vedic Chant for Delta in outputs/shruti_mantra...
Saved Vedic Chant for Delta to outputs/shruti_mantra/Delta_chant.wav
Generating Vedic Chant for Theta in outputs/shruti_mantra...
Saved Vedic Chant for Theta to outputs/shruti_mantra/Theta_chant.wav
Generating Vedic Chant for Alpha in outputs/shruti_mantra...
Saved Vedic Chant for Alpha to outputs/shruti_mantra/Alpha_chant.wav
Generating Vedic Chant for Beta in outputs/shruti_mantra...
Saved Vedic Chant for Beta to outputs/shruti_mantra/Beta_chant.wav
Generating Vedic Chant for Gamma in outputs/shruti_mantra...
Saved Vedic Chant for Gamma to outputs/shruti_mantra/Gamma_chant.wav


In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.io.wavfile import write

# Constants
sample_rate = 44100  # Sample rate for audio
chalan_duration = 108  # Duration for Chalan Swara file (in seconds)
shruti_frequencies = [
    256, 266, 276, 288, 300, 312, 324, 336, 348, 362,
    376, 390, 404, 420, 436, 452, 470, 488, 508, 528,
    548, 568
]

# Function to generate Chalan Swara (combined tones)
def generate_combined_chalan_swara(data_dict, output_folder, file_name, duration):
    print(f"Generating Combined Chalan Swara in {output_folder}...")
    os.makedirs(output_folder, exist_ok=True)

    # Initialize combined audio
    combined_audio = np.zeros(int(sample_rate * duration))
    time_values = np.linspace(0, duration, int(sample_rate * duration))

    # Process each channel and layer its audio
    for channel_name, data in data_dict.items():
        print(f"Processing channel: {channel_name}")
        data_min, data_max = data.min(), data.max()

        # Map channel data to Shruti frequencies
        xp = np.linspace(data_min, data_max, len(shruti_frequencies))
        mapped_shrutis = np.interp(data, xp, shruti_frequencies)

        # Generate channel-specific audio with dynamic variation
        channel_audio = np.zeros(int(sample_rate * duration))
        glide_factor = np.linspace(0, 1, len(mapped_shrutis))  # Smooth glide effect
        for i, freq in enumerate(mapped_shrutis):
            dynamic_freq = freq + 10 * np.sin(2 * np.pi * glide_factor[i] * time_values)  # Micro-fluctuations
            channel_audio += 0.2 * np.sin(2 * np.pi * dynamic_freq * time_values)

        # Add channel audio to combined audio
        combined_audio += channel_audio

    # Normalize the combined audio
    combined_audio /= np.max(np.abs(combined_audio)) if np.max(np.abs(combined_audio)) != 0 else 1

    # Save the combined audio as a WAV file
    output_path = os.path.join(output_folder, file_name)
    write(output_path, sample_rate, combined_audio.astype(np.float32))
    print(f"Saved Combined Chalan Swara to {output_path}")

# Read EEG data
data = pd.read_csv('data/eeg_data.csv', low_memory=False)
data['TimeStamp'] = pd.to_datetime(data['TimeStamp'])
data.set_index('TimeStamp', inplace=True)

# Combine EEG bands
data['Delta'] = data[['Delta_TP9', 'Delta_AF7', 'Delta_AF8', 'Delta_TP10']].sum(axis=1)
data['Theta'] = data[['Theta_TP9', 'Theta_AF7', 'Theta_AF8', 'Theta_TP10']].sum(axis=1)
data['Alpha'] = data[['Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10']].sum(axis=1)
data['Beta'] = data[['Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10']].sum(axis=1)
data['Gamma'] = data[['Gamma_TP9', 'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10']].sum(axis=1)

# Generate Combined Chalan Swara
output_folder = "outputs/shruti_mantra"
file_name = "Chalan_Swara_Mantra.wav"
generate_combined_chalan_swara(
    data_dict={
        'Delta': data['Delta'],
        'Theta': data['Theta'],
        'Alpha': data['Alpha'],
        'Beta': data['Beta'],
        'Gamma': data['Gamma']
    },
    output_folder=output_folder,
    file_name=file_name,
    duration=chalan_duration
)
